In [1]:
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
contract = pd.read_pickle('data/minute_ES_cont_non_active_included.pickle')
multiplier = 50
min_tick = 0.25

In [3]:
contract = contract.loc['20180202':]
contract

,open,high,low,close,volume,average,barCount
date,,,,,,,
2018-02-02 00:00:00,2826.50,2828.25,2826.50,2827.75,2034,2827.275,435
2018-02-02 00:01:00,2827.75,2828.00,2827.00,2827.25,845,2827.375,143
2018-02-02 00:02:00,2827.00,2827.75,2827.00,2827.50,386,2827.200,75
2018-02-02 00:03:00,2827.50,2828.00,2827.25,2827.75,888,2827.550,109
2018-02-02 00:04:00,2827.50,2829.25,2827.50,2829.25,657,2828.500,99
...,...,...,...,...,...,...,...
2019-09-05 08:51:00,2957.50,2957.75,2957.50,2957.50,361,2957.525,70
2019-09-05 08:52:00,2957.50,2958.00,2957.00,2957.25,517,2957.575,112
2019-09-05 08:53:00,2957.50,2957.75,2957.25,2957.50,267,2957.400,85


In [4]:
#periods = 20
periods = [5, 10, 20, 40, 80] #, 160, 320]
ema_fast = 120 # number of periods for moving average filter
sl_atr = 1 # stop loss in ATRs
atr_periods = 80 # number of periods to calculate ATR on
time_int = 15 # interval in minutes to be used to define volume candle
commission = 2.1 # level of commision per transaction in dollars
bankroll = 15000

In [5]:
avg_vol = contract.loc['20190621':].volume.rolling(time_int).sum().mean()
avg_vol

15894.085711955635

In [6]:
avg_vol = 66386

In [7]:
vol_candles = group_by_volume(contract, avg_vol)
vol_candles['atr'] = get_ATR(vol_candles, atr_periods)

In [8]:
vol_candles

,date,open,high,low,close,barCount,volume,atr
label,,,,,,,,
0,2018-02-02 04:40:00,2826.50,2831.00,2811.75,2817.75,13928,66277,19.250000
1,2018-02-02 10:07:00,2817.75,2821.25,2805.75,2807.00,14985,66499,17.351562
2,2018-02-02 11:27:00,2807.25,2810.25,2797.00,2801.00,14005,68352,15.950055
3,2018-02-02 14:08:00,2801.00,2806.75,2800.00,2806.25,13335,66372,13.563081
4,2018-02-02 14:44:00,2806.25,2808.75,2799.50,2808.25,10461,64369,12.656802
...,...,...,...,...,...,...,...,...
8705,2019-09-04 21:58:00,2938.00,2939.00,2937.00,2937.75,5006,46534,8.116252
8706,2019-09-05 02:05:00,2937.50,2942.00,2936.50,2941.00,12977,106671,8.051653
8707,2019-09-05 03:41:00,2941.00,2962.50,2940.75,2959.25,17437,65925,8.389884


In [9]:
data = vol_candles
data.loc[:, 'ema'] = data.close.ewm(span=ema_fast).mean()
#data.loc[:, 'min'] = data.close.rolling(periods).min()
#data.loc[:,'max'] = data.close.rolling(periods).max()
rows_to_drop = max(periods)
#rows_to_drop = 80
rows_to_drop = max(rows_to_drop, ema_fast)
data = data.iloc[rows_to_drop:]
data.reset_index(drop=True, inplace=True)

In [10]:
#data['buy_signal'] = (data.close >= data['max']) * 1
#data['sell_signal'] = (data.close <= data['min']) * -1
#data['signal'] = data.buy_signal + data.sell_signal
#del data['buy_signal']
#del data['sell_signal']
data['signal'] = get_signals(data.close, periods)
data['filter'] = np.sign(data['close'] - data['ema'])
data['filtered_signal'] = data['signal'] * ((data['signal'] * data['filter']) == 1)
#data['cum_position'] = data['position'].cumsum().clip(lower=-1, upper=1)
#data.tail(50)

In [11]:
# while in position maintain open price and transaction direction
data['positions'] = 0
# flag to execute transaction at next data point
data['mark'] = False
# note the reason for transaction at next data point
data['reason'] = ''
# record commission paid
data['commission'] = 0
# record transaction price
data['price'] = 0
# entry price for stop loss calculation
data['entry'] = 0

for item in data.itertuples(): 
    # first row doesn't have to check if we have positions or execute transactions
    if not item.Index == 0:
        # starting position is the same as previous day position
        data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'positions'] 
        data.loc[item.Index, 'entry'] = data.loc[(item.Index - 1), 'entry'] 
        # execute transactions
        if data.loc[(item.Index -  1), 'mark']:
            # close position
            if data.loc[item.Index, 'positions']:
                data.loc[item.Index, 'positions'] = 0
                data.loc[item.Index, 'entry'] = 0
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * np.sign(data.loc[(item.Index - 1), 'entry']) * -1
            # open position
            else:
                data.loc[item.Index, 'positions'] = data.loc[(item.Index - 1), 'signal']
                data.loc[item.Index, 'entry'] = item.open * data.loc[(item.Index - 1), 'signal']
                # record transaction price
                data.loc[item.Index, 'price'] = item.open * data.loc[(item.Index -1), 'signal']
            # record commission paid
            data.loc[item.Index, 'commission'] = commission

    # check for close signal
    if data.loc[item.Index, 'positions'] != 0 and np.sign(item.signal) != 0:
        if np.sign(data.loc[item.Index, 'positions']) != np.sign(item.signal):
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'close'
    # check for stop-loss signal
    if data.loc[item.Index, 'positions'] != 0:
        if data.loc[item.Index, 'positions'] > 0:
            # long positions
            if item.close <= (data.loc[item.Index, 'entry'] - (item.atr * sl_atr)):
                data.loc[item.Index, 'mark'] = True
                data.loc[item.Index, 'reason'] = 'stop-out'
            # short positions
            elif item.close >= (data.loc[item.Index, 'entry'] + (item.atr * sl_atr)):
                data.loc[item.Index, 'mark'] = True
                data.loc[item.Index, 'reason'] = 'stop-out'
    # check for entry signal
    if data.loc[item.Index, 'positions'] == 0:
        if item.filtered_signal != 0:
            data.loc[item.Index, 'mark'] = True
            data.loc[item.Index, 'reason'] = 'entry'
# close any open positions
if data[data.price != 0].price.count() % 2 != 0:
    data.loc[data.index[-1]+1, 'price'] = data.open.iloc[-1] * np.sign(data.entry.iloc[-1]) * -1
    data.loc[data.index[-2]+1, 'entry'] = 0
data.set_index('date', inplace=True)

In [12]:
pnl = data[['open','positions', 'commission', 'mark', 'reason', 'price', 'entry']]
pnl['mark'] = pnl['mark'].shift(1)
pnl['reason'] = pnl['reason'].shift(1)
pnl['cum_pnl'] = pnl['price'].cumsum() - pnl['entry']
pnl['balance'] = bankroll + pnl.cum_pnl * multiplier

In [13]:
pnl

,open,positions,commission,mark,reason,price,entry,cum_pnl,balance
date,,,,,,,,,
2018-02-06 16:25:00,2645.50,0.0,0.0,NaN,NaN,0.00,0.0,0.00,15000.0
2018-02-06 16:31:00,2639.75,0.0,0.0,False,,0.00,0.0,0.00,15000.0
2018-02-06 16:36:00,2634.25,0.0,0.0,False,,0.00,0.0,0.00,15000.0
2018-02-06 16:42:00,2617.50,0.0,0.0,False,,0.00,0.0,0.00,15000.0
2018-02-06 16:48:00,2628.25,0.0,0.0,False,,0.00,0.0,0.00,15000.0
...,...,...,...,...,...,...,...,...,...
2019-09-05 02:05:00,2937.50,1.0,0.0,False,,0.00,2932.5,-583.00,-14150.0
2019-09-05 03:41:00,2941.00,0.0,2.1,True,stop-out,-2941.00,0.0,-591.50,-14575.0
2019-09-05 06:01:00,2959.00,1.0,2.1,True,entry,2959.00,2959.0,-591.50,-14575.0


In [14]:
daily = pnl.resample('D').last()
daily['return'] = daily.balance.pct_change()

In [15]:
pnl.price.sum()

-599.25

In [16]:
pnl.price.sum() *multiplier

-29962.5

In [17]:
pnl.commission.sum()

1778.7

In [18]:
pnl.price.sum()*multiplier - pnl.commission.sum()

-31741.2

In [19]:
pnl[pnl['price'] != 0].price.count() / daily.price.count()

2.0682926829268293

In [20]:
perf_stats(daily['return'])

Annual return               NaN
Cumulative returns    -2.038290
Annual volatility      8.472240
Sharpe ratio           0.361564
Calmar ratio                NaN
Stability                   NaN
Max drawdown          -2.038591
Omega ratio            1.195038
Sortino ratio          0.672596
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.941263
Daily value at risk   -1.055246
dtype: float64

In [21]:
daily.price.count()

410

In [22]:
pnl[pnl.price != 0]

,open,positions,commission,mark,reason,price,entry,cum_pnl,balance
date,,,,,,,,,
2018-02-06 21:09:00,2675.75,1.0,2.1,True,entry,2675.75,2675.75,0.00,15000.0
2018-02-06 22:14:00,2695.00,0.0,2.1,True,stop-out,-2695.00,0.00,-19.25,14037.5
2018-02-07 15:48:00,2705.75,1.0,2.1,True,entry,2705.75,2705.75,-19.25,14037.5
2018-02-07 21:58:00,2688.50,0.0,2.1,True,stop-out,-2688.50,0.00,-2.00,14900.0
2018-02-07 22:14:00,2679.50,-1.0,2.1,True,entry,-2679.50,-2679.50,-2.00,14900.0
...,...,...,...,...,...,...,...,...,...
2019-09-04 09:27:00,2924.00,0.0,2.1,True,close,2924.00,0.00,-583.00,-14150.0
2019-09-04 11:35:00,2932.50,1.0,2.1,True,entry,2932.50,2932.50,-583.00,-14150.0
2019-09-05 03:41:00,2941.00,0.0,2.1,True,stop-out,-2941.00,0.00,-591.50,-14575.0


In [23]:
pnl[pnl.price != 0].count()

open          847
positions     847
commission    847
mark          848
reason        848
price         848
entry         848
cum_pnl       848
balance       848
dtype: int64